
<img align="left" style="padding-right:100px;" src="http://pvexpect.com/Driftsledelse/Dagligvarer/nmbu.png">

  **Digital Driftsledelse - IND310**

 Jesper Ørntoft Frausig, 2022


#**8.0 Interaktive diagrammer**

- [8.1 Interaktivt diagrammer](#8.0)
- [8.1 Interaktivt gjennomsnitt](#8.1)
- [8.2 Interaktivt diagram med kryss-høydepunkt](#8.2)
- [8.3 Interaktivt kryssfilter](#8.3)
- [8.4 Interaktiv legende](#8.4)
- [8.5 Interaktiv rektangulær markering](#8.5)
- [8.6 Interaktivt spredningsdiagram og koblet lagdelt histogram](#8.6)
- [8.7 Multi-Line Highlight](#8.7)
- [8.8 Verktøytips for flere linjer](#8.8)
- [8.9 Spredningsplott med flere paneler med koblet markering](#8.9)
- [8.10 Flere interaksjoner](#8.10)
- [8.11 Spredningsplott og histogram med intervallvalg](#8.11)
- [8.12 Eksempel på utvalgsdetaljer](#8.12)
- [8.13 Utvelgelseshistogram](#8.13)
- [8.14 Enkelt interaktivt farget spredningsdiagram](#8.14)
- [8.15 Bruke utvalgsintervall med mark_area](#8.15)

<a name="8.1"></a>
## **8.1 Interaktivt gjennomsnitt**

https://altair-viz.github.io/gallery/selection_layer_bar_month.html

In [1]:

!pip install pandas_read_xml


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 468 kB 5.1 MB/s 
     |████████████████████████████████| 140 kB 64.9 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:

import pandas_read_xml as pdx
import pandas as pd
url="http://api.sehavniva.no/tideapi.php?lat=59.911491&lon=10.757933&fromtime=2019-01-01T00%3A00&totime=2019-12-31T00%3A00&datatype=all&refcode=cd&place=Oslo&file=&lang=en&interval=60&dst=0&tzone=&tide_request=locationdata"

df = pdx.read_xml(url, ['tide', 'locationdata', 'data'])
df

,0,1,2
0,"{'@type': 'observation', '@unit': 'cm', 'water...","{'@type': 'prediction', '@unit': 'cm', 'waterl...","{'@type': 'weathereffect', '@unit': 'cm', 'wat..."


In [3]:
value=[]
time=[]

for i in pd.DataFrame.from_dict(df[0][0]).waterlevel:

  value.append(i['@value'])
  time.append(i['@time'])

data=pd.concat([pd.Series(value),pd.Series(time)], axis=1)
data.rename(columns={0: 'value', 1:'time'}, inplace=True)

In [6]:
import altair as alt

alt.data_transformers.disable_max_rows()
#source = data.seattle_weather()
brush = alt.selection(type='interval', encodings=['x'])

bars = alt.Chart().mark_bar().encode(
    x='month(time):O',
    y='mean(value):Q',
    opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
).add_selection(
    brush
)

line = alt.Chart().mark_rule(color='firebrick').encode(
    y='mean(value):Q',
    size=alt.SizeValue(3)
).transform_filter(
    brush
)

alt.layer(bars, line, data=data)

alt.LayerChart(...)

<a name="8.2"></a>
## **8.2 Interaktivt diagram med kryss-høydepunkt**

https://altair-viz.github.io/gallery/interactive_cross_highlight.html

In [7]:
import pandas as pd
data = pd.io.stata.read_stata('https://github.com/ECN431/2018/blob/master/lab5/fisheries.dta?raw=true')
data=data.sample(n=5000, random_state=1)
data

,quality,age,tool,preservation,area,condition
29286,20,0.0,Garn,Iset,Skagerrak,Sløyd med hode
43016,12,0.0,Juksa,Fersk/ukonservert,Røstbanken til Malangsgrunnen,"Sløyd uten hode, rundsnitt"
189375,20,0.0,Garn,Fersk/ukonservert,Røstbanken til Malangsgrunnen,Sløyd med hode
131902,12,0.0,Line,Fersk/ukonservert,Røstbanken til Malangsgrunnen,Sløyd med hode
137254,20,0.0,Garn,Fersk/ukonservert,Øst-Finnmark,Sløyd med hode
...,...,...,...,...,...,...
106393,20,0.0,Garn,Fersk/ukonservert,Øst-Finnmark,Rund
123345,20,0.0,Juksa,Fersk/ukonservert,Vestfjorden (Lofoten),Sløyd med hode
30248,10,0.0,Snurrevad,Iset,Eigersundbanken,Sløyd med hode
86339,20,0.0,Garn,Fersk/ukonservert,Røstbanken til Malangsgrunnen,"Sløyd uten hode, rundsnitt"


In [8]:
import altair as alt


pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(data).mark_rect().encode(
    alt.X('quality:Q', bin=True),
    alt.Y('age:Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
)

circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(
    pts
)

bar = alt.Chart(data).mark_bar().encode(
    x='tool:N',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=550,
    height=200
).add_selection(pts)

alt.vconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)

<a name="8.3"></a>
## **8.3 Interaktivt kryssfilter**

https://altair-viz.github.io/gallery/interactive_layered_crossfilter.html

In [9]:
import pandas as pd
data = pd.io.stata.read_stata('https://github.com/ECN431/2018/blob/master/lab5/fisheries.dta?raw=true')
data=data.sample(n=5000, random_state=1)
data

,quality,age,tool,preservation,area,condition
29286,20,0.0,Garn,Iset,Skagerrak,Sløyd med hode
43016,12,0.0,Juksa,Fersk/ukonservert,Røstbanken til Malangsgrunnen,"Sløyd uten hode, rundsnitt"
189375,20,0.0,Garn,Fersk/ukonservert,Røstbanken til Malangsgrunnen,Sløyd med hode
131902,12,0.0,Line,Fersk/ukonservert,Røstbanken til Malangsgrunnen,Sløyd med hode
137254,20,0.0,Garn,Fersk/ukonservert,Øst-Finnmark,Sløyd med hode
...,...,...,...,...,...,...
106393,20,0.0,Garn,Fersk/ukonservert,Øst-Finnmark,Rund
123345,20,0.0,Juksa,Fersk/ukonservert,Vestfjorden (Lofoten),Sløyd med hode
30248,10,0.0,Snurrevad,Iset,Eigersundbanken,Sløyd med hode
86339,20,0.0,Garn,Fersk/ukonservert,Røstbanken til Malangsgrunnen,"Sløyd uten hode, rundsnitt"


In [10]:
import altair as alt

brush = alt.selection(type='interval', encodings=['x'])

# Define the base chart, with the common parts of the
# background and highlights
base = alt.Chart().mark_bar().encode(
    x=alt.X(alt.repeat('column'), type='quantitative', bin=alt.Bin(maxbins=20)),
    y='count()'
).properties(
    width=160,
    height=130
)

# gray background with selection
background = base.encode(
    color=alt.value('#ddd')
).add_selection(brush)

# blue highlights on the transformed data
highlight = base.transform_filter(brush)

# layer the two charts & repeat
alt.layer(
    background,
    highlight,
    data=data
).repeat(column=["quality", "age"])

alt.RepeatChart(...)

<a name="8.4"></a>
## **8.4 Interactive Legend**

https://altair-viz.github.io/gallery/interactive_legend.html

<a name="8.5"></a>
## **8.5 Interaktiv rektangulær markering**

https://altair-viz.github.io/gallery/interactive_brush.html

In [11]:
import altair as alt
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')

brush = alt.selection(type='interval')

alt.Chart(data).mark_circle(size=60).encode(
    x='Req to dissolved oxygen:Q',
    y='Req to Living space:Q',
    color=alt.condition(brush, 'Fisk:O', alt.value('grey')),
    tooltip=['Fisk og fiske art']
).add_selection(brush)

alt.Chart(...)

<a name="8.6"></a>
## **8.6 Interaktivt spredningsdiagram og koblet lagdelt histogram**

In [12]:
import altair as alt
import pandas as pd
import numpy as np

import altair as alt
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')


selector = alt.selection_single(empty='all', fields=['Fisk'])

color_scale = alt.Scale(domain=['M', 'F'],
                        range=['#1FC3AA', '#8624F5'])

base = alt.Chart(data).properties(
    width=250,
    height=250
).add_selection(selector)

points = base.mark_point(filled=True, size=200).encode(
    x=alt.X('mean(Req to dissolved oxygen):Q',
            scale=alt.Scale(domain=[0,10])),
    y=alt.Y('mean(Req to Living space):Q',
            scale=alt.Scale(domain=[0,10])),
    color=alt.condition(selector,
                        'Fisk:N',
                        alt.value('lightgray')),
)

hists = base.mark_bar(opacity=0.5, thickness=100).encode(
    x=alt.X('Pris:Q',
            bin=alt.Bin(step=25), # step keeps bin size the same
            scale=alt.Scale(domain=[150,500])),
    y=alt.Y('count()',
            stack=None,
            scale=alt.Scale(domain=[0,5])),
    color=alt.Color('Fisk:N')
).transform_filter(
    selector
)


points | hists

alt.HConcatChart(...)

<a name="8.7"></a>
## 8.7 Flere linjer med highlight


https://altair-viz.github.io/gallery/multiline_highlight.html

In [13]:
#Installere bibliotekker
!pip install yfinance
!pip install pandas-datareader
!pip install yahoofinancials
!pip install dbnomics
!pip install dask[dataframe]
!pip install d6tstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15192 sha256=b4df409639e81d4fc3b01aae0198c30d8b53ef7865c3956a282fa66f4b69ebc3
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="5y")
  return hist

In [15]:
#Hent aktie symboler
import pandas as pd
url="https://raw.githubusercontent.com/jfrausig/test/master/dagligvarer_ticker.csv" # dk_ticker #trend_bb_4
df=pd.read_csv(url, error_bad_lines=False)
aktier=[]
for index, row in df.iterrows():
    d=row.to_dict()
    aktier.append(d)
aktier=pd.DataFrame(aktier)
aktier['ticker'] = [x.upper() for x in aktier.ticker]




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:

!mkdir stock_dfs

In [17]:
import csv
import os
import yfinance as yf
import pandas as pd
import time
start = time.time()

ticker_list = []

ticker_list=list(aktier.ticker)

data = yf.download(
        tickers = ticker_list,
        period = '2y',
        interval = '1wk',
        group_by = 'ticker',
        auto_adjust = False,
        prepost = False,
        threads = True,
        proxy = None
    )

data = data.T

#for ticker in aktier.ticker:
for i, ticker in enumerate(aktier.ticker): 
    aktie=pd.DataFrame(aktier[aktier.ticker==ticker].Virksomhed)['Virksomhed'][i]
    try:
      data.loc[(ticker,),].T.to_csv('stock_dfs/' + aktie + '.csv', sep=',', encoding='utf-8')
    except:
      print(str(aktie)+" try failed")
print('It took', time.time()-start, 'seconds.')

[*********************100%***********************]  12 of 12 completed

4 Failed downloads:
- SSC.OL: No data found, symbol may be delisted
- ORKLA: No data found, symbol may be delisted
- SALMON.OL: No data found, symbol may be delisted
- SALMAR: No data found, symbol may be delisted
It took 0.8264644145965576 seconds.


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [18]:
  #sammenfør virksomheds info
import glob
import d6tstack.combine_csv
import dask.dataframe as dd
cfg_fnames = list(glob.glob('stock_dfs/*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames)

# check columns
print('all equal',c.is_all_equal())
print('')
print(c.is_column_present())

fnames = d6tstack.combine_csv.CombinerCSV(cfg_fnames).to_csv_align(output_dir='test_data/output/')


# consistent format
virksomheds_info = dd.read_csv('test_data/output/*.csv',dtype={'ask': 'float64',
})
aktie_tidsserie=virksomheds_info.compute()

sniffing columns ok
all equal True

                                           Date  Open  High   Low  Close  Adj Close  Volume
file_path                                                                                  
stock_dfs/Andfjord_Salmon.csv              True  True  True  True   True       True    True
stock_dfs/Austevoll Seafood.csv            True  True  True  True   True       True    True
stock_dfs/Bakkafrost.csv                   True  True  True  True   True       True    True
stock_dfs/Grieg Seafood.csv                True  True  True  True   True       True    True
stock_dfs/Lerøy Seafood Group.csv          True  True  True  True   True       True    True
stock_dfs/Mowi.csv                         True  True  True  True   True       True    True
stock_dfs/Norway Royal Salmon.csv          True  True  True  True   True       True    True
stock_dfs/Orkla Group.csv                  True  True  True  True   True       True    True
stock_dfs/SalMar.csv                       T

In [19]:
import altair as alt



base = alt.Chart(aktie_tidsserie).properties(width=550)

line = base.mark_line().encode(
    x='Date:O',
    y='Close',
    color='filename'
)

rule = base.mark_rule().encode(
    y='average(Close)',
    color='filename',
    size=alt.value(2)
)

line + rule

alt.LayerChart(...)

In [20]:
import altair as alt

highlight = alt.selection(type='single', on='mouseover',
                          fields=['filename'], nearest=True)

base = alt.Chart(aktie_tidsserie).encode(
    x='Date:O',
    y='Close',
    color='filename:N'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)

points + lines

alt.LayerChart(...)

<a name="8.8"></a>
## **8.8 Verktøytips for flere linjer**

https://altair-viz.github.io/gallery/multiline_tooltip.html

In [21]:
aktie_tidsserie

,Date,Open,High,Low,Close,Adj Close,Volume,filepath,filename
0,2020-10-26,43.500000,46.685001,40.014999,45.000000,45.000000,63505.0,stock_dfs/Andfjord_Salmon.csv,Andfjord_Salmon.csv
1,2020-11-02,45.994999,46.880001,45.000000,46.000000,46.000000,26489.0,stock_dfs/Andfjord_Salmon.csv,Andfjord_Salmon.csv
2,2020-11-09,46.000000,47.650002,43.605000,44.400002,44.400002,74307.0,stock_dfs/Andfjord_Salmon.csv,Andfjord_Salmon.csv
3,2020-11-16,44.000000,46.200001,43.500000,45.994999,45.994999,64263.0,stock_dfs/Andfjord_Salmon.csv,Andfjord_Salmon.csv
4,2020-11-23,46.500000,47.250000,45.000000,45.000000,45.000000,86891.0,stock_dfs/Andfjord_Salmon.csv,Andfjord_Salmon.csv
...,...,...,...,...,...,...,...,...,...
101,2022-10-03,1.088000,1.270000,1.050000,1.270000,1.270000,28621.0,stock_dfs/atlantic sapphire.csv,atlantic sapphire.csv
102,2022-10-10,1.240000,1.400000,1.150000,1.270000,1.270000,40101.0,stock_dfs/atlantic sapphire.csv,atlantic sapphire.csv
103,2022-10-17,0.697700,0.939000,0.438800,0.490000,0.490000,445845.0,stock_dfs/atlantic sapphire.csv,atlantic sapphire.csv
104,2022-10-24,0.584500,0.682100,0.568400,0.682100,0.682100,80645.0,stock_dfs/atlantic sapphire.csv,atlantic sapphire.csv


In [22]:
import altair as alt
import pandas as pd
import numpy as np

np.random.seed(42)

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')



# The basic line
line = alt.Chart(aktie_tidsserie).mark_line(interpolate='basis').encode(
    x='Date:O',
    y='Close:Q',
    color='filename:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(aktie_tidsserie).mark_point().encode(
    x='Date:O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Close:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(aktie_tidsserie).mark_rule(color='gray').encode(
    x='Date:O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

<a name="8.9"></a>
## **8.9 Spredningsplott med flere paneler med koblet markering**

https://altair-viz.github.io/gallery/scatter_linked_brush.html

In [ ]:
"""
import altair as alt
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')
data.head(5)

alt.Chart(data).mark_circle(size=60).encode(
    x='Req to dissolved oxygen:Q',
    y='Req to Living space:Q',
    color='Fisk',
    size='Pris',
    tooltip=['Fisk og fiske art']
).interactive()
"""

In [23]:
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')
data

,Fisk,Fiske art,Fisk og fiske art,Req to dissolved oxygen,Req to Living space,Req to Acoustic comfort,Req to Fish waste free,Req to Food waste free,Req to Parasite free,LV 1 - Dag,...,LV 3 - Grensverdi,LV 3 - Vekst per time,LV 4 - Dag,LV 4 - Grensverdi,LV 4 - Vekst per time,Waste produced,Oxygen needed,Seed weight,Pris,Antall
0,Tilipa,Common tilipa,Tilipa - Common tilipa,2.8,2.2,2.7,1.9,1.9,2.9,0,...,0.22,1 %,8,0.75,0 %,0.03,0.03,0.1,155,5
1,Tilipa,Three spotted tilipa,Tilipa - Three spotted tilipa,2.8,2.8,2.8,1.9,1.9,3.1,0,...,0.22,1 %,8,0.75,0 %,0.03,0.03,0.1,165,5
2,Tilipa,Mozambique tilipa,Tilipa - Mozambique tilipa,2.8,3.0,2.8,1.9,2.2,3.1,0,...,0.22,1 %,8,0.75,0 %,0.03,0.03,0.1,175,5
3,Tilipa,Nile tilipa,Tilipa - Nile tilipa,2.8,4.0,2.8,2.2,2.2,3.8,0,...,0.22,1 %,10,0.75,0 %,0.03,0.03,0.1,195,5
4,Tilipa,Blue tilipa,Tilipa - Blue tilipa,3.0,4.5,3.0,3.0,3.0,4.0,0,...,0.22,1 %,10,0.75,0 %,0.03,0.03,0.1,215,5
5,Snakehead,Forest snakehead,Snakehead - Forest snakehead,3.5,3.5,3.5,2.2,2.2,3.5,0,...,0.22,1 %,8,0.75,0 %,0.04,0.06,0.1,212,4
6,Snakehead,Northern snakehead,Snakehead - Northern snakehead,3.5,4.0,3.5,2.2,2.2,4.0,0,...,0.22,1 %,8,0.75,0 %,0.04,0.06,0.1,220,4
7,Snakehead,Borna snakehead,Snakehead - Borna snakehead,3.5,4.5,3.5,2.2,2.7,4.0,0,...,0.22,1 %,8,0.75,0 %,0.04,0.06,0.1,232,4
8,Snakehead,Snakehead Murrel,Snakehead - Snakehead Murrel,3.6,5.0,3.6,2.9,2.9,4.5,0,...,0.22,1 %,8,0.75,0 %,0.04,0.06,0.1,244,4
9,Snakehead,Emperor snakehead,Snakehead - Emperor snakehead,3.7,5.3,3.6,3.1,3.0,5.0,0,...,0.22,1 %,10,0.75,0 %,0.04,0.06,0.1,264,4


In [24]:
import altair as alt
import pandas as pd

data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')


brush = alt.selection(type='interval', resolve='global')

base = alt.Chart(data).mark_point().encode(
    y='Req to dissolved oxygen:Q',
    color=alt.condition(brush, 'Fisk:N', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=250,
    height=250
)

diagram=base.encode(x='Req to Living space:Q') | base.encode(x='Req to Acoustic comfort:Q')

diagram #.save("Interaktivt diagram.html")

alt.HConcatChart(...)

<a name="8.10"></a>
## **8.10 Flere interaksjoner**

https://altair-viz.github.io/gallery/multiple_interactions.html

<a name="8.11"></a>
##**8.11 Spredningsplott og histogram med intervallvalg**

https://altair-viz.github.io/gallery/scatter_with_histogram.html

<a name="8.12"></a>
##**8.12 Eksempel på utvalgsdetaljer**

https://altair-viz.github.io/gallery/select_detail.html

<a name="8.13"></a>
## **8.13 Utvelgelseshistogram**

https://altair-viz.github.io/gallery/selection_histogram.html

<a name="8.14"></a>
##**8.14 Enkelt interaktivt farget spredningsdiagram**

https://altair-viz.github.io/gallery/interactive_scatter_plot.html

<a name="8.15"></a>
##**8.15 Bruke utvalgsintervall med mark_area**

https://altair-viz.github.io/gallery/select_mark_area.html